<a href="https://colab.research.google.com/github/Moriblo/MVP_Spt4_EngSoft/blob/main/MVP_SPT4_EngSoft_MLFI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Engenharia de Software Inteligentes - MVP Sprint 4

## Coleta e Análise de Dados

### Requirements

In [52]:
!pip install gdown

In [62]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import gdown
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from google.colab import files

### Carga dos Datasets

In [94]:
# Informa a URL de importação do dataset
id_arquivo_1 = '1R8vASJ_vSzykspidSdAVEXvnWIW8gIiG'
id_arquivo_2 = '1eFlNnCsX8HtRjAgPA2IsM-9nzKwG3FLD'

# URLs para download direto dos arquivos
url_1 = f'https://drive.google.com/uc?id={id_arquivo_1}'
url_2 = f'https://drive.google.com/uc?id={id_arquivo_2}'

# Faça o download dos arquivos
gdown.download(url_1, 'arquivo1.csv', quiet=False)
gdown.download(url_2, 'arquivo2.csv', quiet=False)

# Lê o arquivo
df_1 = pd.read_csv('arquivo1.csv', delimiter=';')
df_2 = pd.read_csv('arquivo2.csv', delimiter=';')

Downloading...
From: https://drive.google.com/uc?id=1R8vASJ_vSzykspidSdAVEXvnWIW8gIiG
To: /content/arquivo1.csv
100%|██████████| 46.2M/46.2M [00:00<00:00, 251MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eFlNnCsX8HtRjAgPA2IsM-9nzKwG3FLD
To: /content/arquivo2.csv
100%|██████████| 49.0M/49.0M [00:00<00:00, 210MB/s]


### Tratando os daframe_1 e dataframe_2

In [95]:
# Adequando VL_QUOTA à quantidade de casas decimais informada no Dicionário de Dados do dataset de origem
df_1['VL_QUOTA'] = df_1['VL_QUOTA'].str.replace('.', '')
df_1['VL_QUOTA'] = df_1['VL_QUOTA'].astype(float)
df_1['VL_QUOTA'] = df_1['VL_QUOTA'] / (10**12)

df_2['VL_QUOTA'] = df_2['VL_QUOTA'].str.replace('.', '')
df_2['VL_QUOTA'] = df_2['VL_QUOTA'].astype(float)
df_2['VL_QUOTA'] = df_2['VL_QUOTA'] / (10**12)

# Removendo as tuplas em que VL_QUOTA seja negativo
df_1[df_1["VL_QUOTA"] >= 0]
df_2[df_2["VL_QUOTA"] >= 0]

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,02/10/2023,1117290.14,33.503627,1121717.33,0.0,0.00,1
1,FI,00.017.024/0001-53,03/10/2023,1117817.51,33.517100,1122168.38,0.0,0.00,1
2,FI,00.017.024/0001-53,04/10/2023,1118356.25,33.530557,1122618.95,0.0,0.00,1
3,FI,00.017.024/0001-53,05/10/2023,1118895.00,33.546346,1123147.56,0.0,0.00,1
4,FI,00.017.024/0001-53,06/10/2023,1116529.26,33.560087,1120766.21,0.0,2841.41,1
...,...,...,...,...,...,...,...,...,...
537273,FI,97.929.213/0001-34,25/10/2023,82084885.53,11.034508,83854769.84,0.0,0.00,2
537274,FI,97.929.213/0001-34,26/10/2023,82347284.41,11.068967,84116640.54,0.0,0.00,2
537275,FI,97.929.213/0001-34,27/10/2023,82233449.05,11.053918,84002275.71,0.0,0.00,2
537276,FI,97.929.213/0001-34,30/10/2023,82026218.20,11.026579,83794515.96,0.0,0.00,2


### Realizando o Join dos dataframes 1 e 2 no dataframe_3, o cálculo de retorno e tratando o dataframe_3

In [96]:
# Considerando somente os dígitos de DIA em relação a DT_COMPTC (Data de competência do documento)
df_1['DT_COMPTC'] = df_1['DT_COMPTC'].str.slice(0, 2)
df_2['DT_COMPTC'] = df_2['DT_COMPTC'].str.slice(0, 2)

# Definindo o index para o JOIN
df_1.set_index(['CNPJ_FUNDO', 'DT_COMPTC'], inplace=True)
df_2.set_index(['CNPJ_FUNDO', 'DT_COMPTC'], inplace=True)

# Juntando os dois DataFrames usando o índice
df_3 = df_2.join(df_1, rsuffix='_1')

# Calculando o retorno, dia a dia, do mês analisado (dataset_2) em relação ao mês anterior (dataset_1) na coluna RETORNO com cinco casas decimais
df_3['RETORNO'] = ((df_3['VL_QUOTA'] - df_3['VL_QUOTA_1']) / df_3['VL_QUOTA']).round(5)

# Mantém somente as colunas referentes ao mês corrente
df_3 = df_3.filter(regex='[^_1]$')

# Removendo todas as tuplas com RETORNO = '' ou '#NOME?'
df_3 = df_3.replace(['#NOME?', ''], np.nan)
df_3 = df_3.dropna(subset=['RETORNO'])

# Removendo todas as tuplas com NR_COTST (Número de Cotistas) > 1
df_3['NR_COTST'] = df_3['NR_COTST'].astype(float)
df_3[df_3["NR_COTST"] > 1]

# Printando e salvando o dataset de resultado
# df_3.to_csv('df_3.csv', sep=';', index=True)
# files.download('dataset_3.csv')

# print(df_3.head())

TP_FUNDO     VL_TOTAL   VL_QUOTA  VL_PATRIM_LIQ  \
CNPJ_FUNDO         DT_COMPTC                                                   
00.068.305/0001-35 04              FI  38152474.85  34.475447    38144200.34   
                   05              FI  38171631.91  34.490780    38161016.68   
                   06              FI  38190005.64  34.505339    38177124.91   
                   11              FI  38218373.42  34.549179    38212542.73   
                   13              FI  38236234.46  34.563357    38228058.82   
...                               ...          ...        ...            ...   
97.929.213/0001-34 19              FI  83654348.17  11.012560    83687985.14   
                   20              FI  83696483.82  11.013358    83694045.16   
                   25              FI  82084885.53  11.034508    83854769.84   
                   26              FI  82347284.41  11.068967    84116640.54   
                   27              FI  82233449.05  11.053918    84002275.71   

                              CAPTC_DIA  RESG_DIA  NR_COTST  RETORNO  
CNPJ_FUNDO         DT_COMPTC                                          
00.068.305/0001-35 04               0.0      0.00    6062.0  0.00931  
                   05               0.0    148.48    6061.0  0.00934  
                   06               0.0      0.00    6061.0  0.00928  
                   11               0.0  13087.65    6060.0  0.00967  
                   13               0.0    164.77    6059.0  0.00916  
...                                 ...       ...       ...      ...  
97.929.213/0001-34 19               0.0      0.00       2.0 -0.01166  
                   20               0.0      0.00       2.0 -0.01083  
                   25               0.0      0.00       2.0 -0.00663  
                   26               0.0      0.00       2.0  0.00085  
                   27               0.0      0.00       2.0  0.00160  

[150827 rows x 8 columns]

## Separação em conjunto de treino e conjunto de teste com holdout

In [44]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e teste
array = dataset.values
X = array[:,0:8]
y = array[:,8]
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação

NameError: ignored

## Modelagem e Inferência

### Criação e avaliação de modelos: linha base

In [ ]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Listas para armazenar os resultados
results = []
names = []

# Avaliação dos modelos
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(15,10))
fig.suptitle('Comparação dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

### Criação e avaliação de modelos: dados padronizados e normalizados

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []


# Criando os elementos do pipeline

# Algoritmos que serão utilizados
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())


# Montando os pipelines

# Dataset original
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))

# Dataset Padronizado
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))

# Dataset Normalizado
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

### Otimização dos hiperparâmetros

In [ ]:
# Tuning do KNN

np.random.seed(7) # definindo uma semente global para este bloco

pipelines = []

# Definindo os componentes do pipeline
knn = ('KNN', KNeighborsClassifier())
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

pipelines.append(('knn-orig', Pipeline(steps=[knn])))
pipelines.append(('knn-padr', Pipeline(steps=[standard_scaler, knn])))
pipelines.append(('knn-norm', Pipeline(steps=[min_max_scaler, knn])))

param_grid = {
    'KNN__n_neighbors': [1,3,5,7,9,11,13,15,17,19,21],
    'KNN__metric': ["euclidean", "manhattan", "minkowski"],
}

# Prepara e executa o GridSearchCV
for name, model in pipelines:
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train)
    # imprime a melhor configuração
    print("Sem tratamento de missings: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_))

## Finalização do Modelo

In [ ]:
# Avaliação do modelo com o conjunto de testes

# Preparação do modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = KNeighborsClassifier(metric='manhattan', n_neighbors=17)
model.fit(rescaledX, y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))

In [ ]:
# Preparação do modelo com TODO o dataset
scaler = StandardScaler().fit(X) # ajuste do scaler com TODO o dataset
rescaledX = scaler.transform(X) # aplicação da padronização com TODO o dataset
model.fit(rescaledX, y)

## Simulando a aplicação do modelo em dados não vistos

In [ ]:
# Novos dados - não sabemos a classe!
data = {'preg':  [1, 9, 5],
        'plas': [90, 100, 110],
        'pres': [50, 60, 50],
        'skin': [30, 30, 30],
        'test': [100, 100, 100],
        'mass': [20.0, 30.0, 40.0],
        'pedi': [1.0, 2.0, 1.0],
        'age': [15, 40, 40],
        }

atributos = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,0:8].astype(float)

# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

In [ ]:
# Predição de classes dos dados de entrada
saidas = model.predict(rescaledEntradaX)
print(saidas)